#1. XPath
* XPath是一门语言<br/>
* XPath可以在XML文档中查找信息<br/>
* XPath支持HTML<br/>
* XPath可以通过元素和属性来进行导航<br/>
* XPath可以用来提取信息<br/>
* XPath比正则表达式更加简单、更加厉害

###安装XPath
安装 lxml 模块<br/>
from lxml import etree<br/>
selector = etree.HTML(源代码)<br/>
selector.xpath(自定义)

###XPath与HTML结构
* 树状结构<br/>
* 逐层展开<br/>
* 逐层定位<br/>
* 寻找独立节点

###基本语法规则
//  定位根节点<br/>
/  往下层寻找<br/>
/text()  提取文本内容<br/>
/@XXX  提取属性内容

In [51]:
from lxml import etree
f = open('D:/Python/crawler/information.html', 'r')
html = f.read()
f.close()
selector = etree.HTML(html)
# 不加指定地提取内容
content = selector.xpath('//ul/li/text()')
print type(content)
for each in content:
    print each

<type 'list'>
这是第一条信息
这是第二条信息
这是第三条信息
不需要的信息1
不需要的信息1
不需要的信息1


In [52]:
# 根据属性提取指定内容
content = selector.xpath('//ul[@id="useful"]/li/text()')
for each in content:
    print each

这是第一条信息
这是第二条信息
这是第三条信息


In [53]:
# 进一步进行指定
# 和group类似，这里是li[1]而不是li[0]
content = selector.xpath('//ul[@id="useful"]/li[1]/text()')
print content[0]

这是第一条信息


In [54]:
# 提取属性1
link = selector.xpath('//a/@href')
for each in link:
    print each

http://www.bbc.com/
http://www.bbc.com/news/world-us-canada-40590094


In [55]:
# 提取属性2
title = selector.xpath('//a/@title')[0]
print title

BBC新闻


###高级运用规则
**div[starts-with(@attribute, "attribute-name")]**<br/>
**div.xpath('string(.)')**

In [56]:
f = open('D:/Python/crawler/infor2.html', 'r')
html = f.read()
f.close()
selector = etree.HTML(html)
content = selector.xpath('//div[starts-with(@id, "test")]/text()')
for each in content:
    print each

需要的内容1
需要的内容2
需要的内容3


In [57]:
f = open('D:/Python/crawler/infor3.html', 'r')
html = f.read()
f.close()
selector = etree.HTML(html)
text_field = selector.xpath('//body/div[@id="test3"]/text()')[0]
print text_field.replace('\n', '').replace(' ', '')
info = selector.xpath('//div[@id="test3"]')[0].xpath('string(.)')
content = info.replace('\n', '').replace(' ', '')
print content

我左青龙，
我左青龙，右白虎，上朱雀，下玄武。老牛在当中，龙头在胸口。


#2. Python多线程

###特点：
多个线程同时处理任务<br/>
高效<br/>
快速

###map函数的使用：
map函数包含序列操作、参数传递、结果保存等一系列操作<br/>
from multipricessing.dummy import Pool<br/>
pool = Pool(4)<br/>
results = pool.map(爬取函数，目标网址列表)

> 引申：map 与 filter<br/>
> map会将一个函数映射到一个输入列表的所有元素上<br/>
> filter进行筛选<br/>
> 两者都要结合lambda<br/>

In [58]:
items = [1, 2, 3, 4, 5]
s = []
for i in items:
    s.append(i ** 2)
print s

[1, 4, 9, 16, 25]


In [59]:
s_1 = list(map(lambda x : x ** 2, items))
print s_1

[1, 4, 9, 16, 25]


In [60]:
# 不仅用于一列表的所有元素输入，甚至可以用于一列表的函数
def mul(x):
    return x * x
def add(x):
    return x + x
funcs = [mul, add]
for i  in range(5):
    value = list(map(lambda x : x(i), funcs))
    print value

[0, 0]
[1, 2]
[4, 4]
[9, 6]
[16, 8]


In [61]:
num = range(-5, 5)
less = list(filter(lambda x : x > 0, num))
less

[1, 2, 3, 4]

###多线程爬虫原理
* requests获取网页<br/>
* XPath提取内容<br/>
* map实现多线程<br/>
<br/>
* 提示：如果出现中文乱码，建议先把保存在本地的文件用Notepad++打开，在“格式”中选择**“以UTF-8格式编码”**，保存后执行可恢复正常

In [70]:
# coding=utf-8
from lxml import etree
import json

html = open('D:/Python/crawler/tieba.html', 'r').read()
selector = etree.HTML(html)
content_field = selector.xpath('//ul/li[@class="j_thread_list clearfix"]')
for each in content_field:
    data_field = json.loads(each.xpath('@data-field')[0].replace('&quot', ''))
    author_name = data_field['author_name']
    user_id = data_field['id']
    reply_num = data_field['reply_num']
    title = each.xpath('div[@class="t_con cleafix"]/div[@class="threadlist_li_right j_threadlist_li_right "]/div\
    [@class="threadlist_lz clearfix"]/div[@class="threadlist_title pull_left j_th_tit"]/a/text()')
    time = each.xpath('div[@class="t_con cleafix"]/div[@class="threadlist_li_right j_threadlist_li_right "]/div\
    [@class="threadlist_detail clearfix"]/div[@class="threadlist_author"]/span[@class="threadlist_reply_date pull_right j_reply_data"]/text()')[0].lstrip(' ')
    if len(title) > 0:
        title_content = title[0]
    else:
        title_content = ' '
    print str(author_name.encode('utf-8')) + ', ' + str(user_id) + ', ' + str(reply_num) + ', ' + str(title_content.encode('utf-8')) + ', ' + str(time)

In [63]:
from lxml import etree
import requests, json

class spider(object):
    def __init__(self):
        print '正在抓取……'
    
    def true_spider(self, url):
        html = requests.get(url)
        selector = etree.HTML(html.text)
        content_field = selector.xpath('//li[@class="j_thread_list clearfix"]')
        for each in content_field:
            data_field = json.loads(each.xpath('@data-field')[0].replace('&quot', ''))
            author_name = data_field['author_name']
            user_id = data_field['id']
            reply_num = data_field['reply_num']
            if each.xpath('div[@class="t_con cleafix"]/div[@class="threadlist_li_right j_threadlist_li_right "]/div\
            [@class="threadlist_lz clearfix"]/div/@class')[0] == 'threadlist_text threadlist_title j_th_tit  ':
                title = each.xpath('div[@class="t_con cleafix"]/div[@class="threadlist_li_right j_threadlist_li_right "]/div\
                [@class="threadlist_lz clearfix"]/div[@class="threadlist_text threadlist_title j_th_tit  "]/a/@title')[0]
                time =each.xpath('div[@class="t_con cleafix"]/div[@class="threadlist_li_right j_threadlist_li_right "]/div\
                [@class="threadlist_detail clearfix"]/div[@class="threadlist_author"]/span[@class="threadlist_reply_date j_reply_data"]/text()')
                if len(time) == 0:
                    time = ' '
                else:
                    time = time[0].lstrip(' ')
            else:
                title = each.xpath('div[@class="t_con cleafix"]/div[@class="threadlist_li_right j_threadlist_li_right "]/div\
                [@class="threadlist_lz clearfix"]/div[@class="threadlist_text threadlist_title j_th_tit  member_thread_title_frs  "]/a/@title')[0]
                time = each.xpath('div[@class="t_con cleafix"]/div[@class="threadlist_li_right j_threadlist_li_right "]/div[@class="threadlist_detail clearfix"]/div\
                [@class="threadlist_author"]/span[@class="threadlist_reply_date j_reply_data"]/text()')
                if len(time) == 0:
                    time = ' '
                else:
                    time = time[0].lstrip(' ')
            print str(author_name.encode('utf-8')) + ', ' + str(user_id) + ', ' + str(reply_num) + ', ' + str(title.encode('utf-8')) + ', ' + str(time)

if __name__ == "__main__":
    tiebaspider = spider()
    for i in range(0, 50, 50):
        new_page = 'http://tieba.baidu.com/f?kw=python&ie=utf-8&pn=' + str(i)
        tiebaspider.true_spider(new_page)

正在抓取……


In [64]:
from multiprocessing.dummy import Pool as ThreadPool
import requests, time

def getsource(url):
    html = requests.get(url)
urls = []
for i in range(1, 11):
    newpage = 'http://tieba.baidu.com/p/3522395718?pn=' + str(i)
    urls.append(newpage)
time1 = time.time()
for i in urls:
    getsource(i)
time2 = time.time()
print 'Single thread costs: ' + str(time2 - time1)

pool = ThreadPool(4)
time3 = time.time()
results = pool.map(getsource, urls)
pool.close()
pool.join()
time4 = time.time()
print 'Multi threads costs: ' + str(time4 - time3)

Single thread costs: 5.72699999809
Multi threads costs: 1.60899996758
